In [ ]:
pip install hyperopt==0.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 
     |████████████████████████████████| 199 kB 51.9 MB/s 
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score
import tensorflow
import keras
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
from pickle import dump
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial, anneal, STATUS_OK
from hyperopt.early_stop import no_progress_loss

In [ ]:
def getdf():
    formatted_file= "/content/drive/MyDrive/preprocessed_data_updated.xlsx"
    df = pd.read_excel(formatted_file,usecols=[1,2,3,4,5])
    return df

def joindf(df):
  cols=['Day','Work']
  df['Day'] = df['Day'].astype(int)
  df['DayWork'] = df[cols].apply(lambda row: 'x'.join(row.values.astype(str)), axis=1)
  group_lot = df.groupby("LOT")
  seq_df = group_lot["DayWork"].apply(list)
  return seq_df
  
def replace_df(seq_df):
  datadf=pd.DataFrame()
  index=0
  for i,val in enumerate (seq_df.index):
    if(len(seq_df[val])==1):
      seq_df=seq_df.drop(val)
    else:
      #seq_df[val]=str(seq_df[val])
      seq_df[val]=str(seq_df[val])
      seq_df[val]=seq_df[val].replace(",","").replace("'","").replace("[","").replace("]","")
      data_ele = pd.DataFrame({'Sequence':seq_df[val]},index=[index])
      datadf=pd.concat([datadf,data_ele])
      index=index+1
        

  ldf = datadf[datadf['Sequence'].str.contains('廃棄')]
  idf = ldf[ldf['Sequence'].str.contains('1x接種')]
  datadf=idf
  return datadf

def load_seq(df):
  seqlist=[]
  for seq in df['Sequence']:
    seqlist.append(seq)
  return seqlist


def to_vocabulary(sequences):
  all_desc = set()
  for d in sequences:
     all_desc.update(d.split())
  return all_desc


def load_clean_seq(data):
  seqlist = load_seq(data)
  sequences=list()
  for seq in seqlist:
    s="startseq "+seq+" endseq"
    sequences.append(s)
  return sequences


def create_tokenizer(sequences):
  print(sequences)
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(sequences)
  return tokenizer


def create_sequences(tokenizer, max_length, descriptions,vocab_size):
  X2, y = list(), list()
	# walk through each image identifier
  for desc in descriptions:
    seq = tokenizer.texts_to_sequences([desc])[0]
    # split one sequence into multiple X,y pairs
    for i in range(1, len(seq)):
      # split into input and output pair
      in_seq, out_seq = seq[:i], seq[i]
      # pad input sequence
      in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
      # encode output sequence
      out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
      # store
      X2.append(in_seq)
      y.append(out_seq)
  return array(X2), array(y)

def max_length(sequences):
	return max(len(d.split()) for d in sequences)

In [ ]:
df = getdf()
seqdf =  joindf(df)
datadf= replace_df(seqdf)
sequences = load_seq(datadf)
vocabulary = to_vocabulary(sequences)


# prepare tokenizer
tokenizer = create_tokenizer(sequences)
print(tokenizer)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
max_len = max_length(sequences)

['1x接種 91xCUT 97xH 98xH 99xH 100xH 101xH 105xH 107xH 109xW 110xWM 115xH 116xH 117xH 118xH 119xH 120xH 121xH 122xH 123xH 124xH 126xH 127xH 129xW 130xWM 134xH 135xH 136xH 137xH 138xH 139xH 140xH 143xH 145xH 146xH 147xW 148xWM 153xH 154xH 155xH 156xH 157xH 158xH 159x廃棄', '1x接種 89xCUT 95xH 96xH 97xH 98xH 99xH 114xS 115xS 116xS 117xS 118xS 119xS 120xW 121xS 125xH 126xH 127xH 128xH 129xH 130xH 132xH 133xH 134xH 135xH 136x廃棄', '1x接種 94xCUT 95xS 96xS 97xS 99xH 100xH 101xH 102xH 103xH 104xH 105xH 107xS 108xS 109xS 110xS 111xS 112xH 115xH 116xH 117xS 118xS 119xS 120xS 121xS 122xH 123xH 124xH 125xH 126xH 127xW 128xS 130xH 131xH 132xH 133xH 134xH 135xH 137xH 138xH 139xH 140xH 142xW 143xS 144xS 147xH 148xH 149xH 151xH 152xH 157xH 160xH 166xH 173xW 174xWM 175xS 180xH 181xH 182xH 183xH 184xH 186xH 190xS 193xH 196xH 205xH 212x廃棄', '1x接種 89xCUT 94xH 95xH 96xH 97xH 98xH 99xS 100xS 101xS 103xW 104xWM 105xS 106xS 107xS 108xS 109xS 110xH 111xH 112xH 113xH 114xH 115xH 116xH 118xS 119xS 120xS 122xW 123xWM 12

In [ ]:
print(max_len)

89


In [ ]:
traindf=datadf.sample(frac=0.8,random_state=200) #random state is a seed value
testdf=datadf.drop(traindf.index)

In [ ]:
# train dataset
train_sequences = load_clean_seq(traindf)
print('Sequences: train=%d' % len(train_sequences))
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
# max_length = max_length_fun(train_sequences)
print('Sequence Length: %d' % max_len)
# prepare sequences
X2train, ytrain = create_sequences(tokenizer, max_len,train_sequences,vocab_size)
 

Sequences: train=348
Vocabulary Size: 582
Sequence Length: 89


In [ ]:
# test dataset
test_sequences = load_clean_seq(testdf)
print('Sequences: train=%d' % len(test_sequences))
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# # determine the maximum sequence length
# max_length = max_length_fun(test_sequences)
print('Sequence Length: %d' % max_len)
# prepare sequences
X2test, ytest = create_sequences(tokenizer, max_len,test_sequences,vocab_size)

Sequences: train=87
Vocabulary Size: 582
Sequence Length: 89


In [ ]:
print(f"X_train : {X2train}")
print("\n")
print(f"y_train: {ytrain}")

X_train : [[  0   0   0 ...   0   0   1]
 [  0   0   0 ...   0   1 120]
 [  0   0   0 ...   1 120  19]
 ...
 [  0   0   0 ... 211 265 247]
 [  0   0   0 ... 265 247 282]
 [  0   0   0 ... 247 282 325]]


y_train: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
space = {
            'optimizer': hp.choice('optimizer',['adam','SGD','RMSprop','Adagrad','Adadelta','Adamax','Nadam']),
            'loss': hp.choice('loss',['categorical_crossentropy','mean_squared_error','mae']),
            'units': hp.uniform('units',128,1024),
            'vocab_size' : vocab_size,
            'max_len' : max_len,
        }

In [ ]:
def define_model(space):
  #sequence model
  inputs2 = Input(shape=(space['max_len'],))
  se1 = Embedding(space['vocab_size'], 256, mask_zero=True)(inputs2)
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(int(space['units']))(se2)
  # decoder model
  decoder2 = Dense(256, activation='relu')(se3)
  outputs = Dense(space['vocab_size'], activation='softmax')(decoder2)
  # tie it together [image, seq] [word]
  model = Model(inputs=inputs2, outputs=outputs)
  model.compile(loss=space['loss'], optimizer=space['optimizer'],metrics=['accuracy'])
  return model

In [ ]:
def hyperparameter_tuning(space):
        clf = define_model(space)
        history = clf.fit(X2train, ytrain,
                          epochs=100,
                          #batch_size=batch_size,
                          verbose=0,
                          validation_split=0.1,
                          shuffle=False,
                          callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,mode="min")]
                          )
        try:
            opt_score = history.history["val_loss"][-1]
            return {"loss": opt_score, "status": STATUS_OK}
        except:
            return {"loss": opt_score, "status": "Failed"}

In [ ]:
trials = Trials()

In [ ]:
best = fmin(fn=hyperparameter_tuning,
                    space=space,
                    algo=tpe.suggest,
                    max_evals=20,
                    trials=trials,
                    early_stop_fn=no_progress_loss(iteration_stop_count=10,
                                                   percent_increase=0.0),
                    return_argmin=False)
print("Best: {}".format(best))